In [1]:
import pandas as pd
import numpy as np

import os

import warnings
warnings.filterwarnings(action='ignore')

import seaborn as sns
import matplotlib.pyplot as plt
import missingno as msno

## 아파트 & 오피스텔

### 집 종류별, estate 데이터 불러오는 함수

In [2]:
def data_load(house_type):
    DATA_DIR = './data/'
    
    folder_path = DATA_DIR + '{}/'.format(house_type)

    file_names = os.listdir(folder_path)
    estate_list = []
    house_list = []

    for file_name in file_names:
        if 'estate' in file_name:
            estate_list.append(file_name)
        else:
            house_list.append(file_name)
            
    estate_all_dataframes = []
    for estate in estate_list:
        df = pd.read_csv(folder_path+estate)
        estate_all_dataframes.append(df)

    estate_df = pd.concat(estate_all_dataframes, ignore_index=True)
    
    house_all_dataframes = []
    for house in house_list:
        df = pd.read_csv(folder_path+house)
        house_all_dataframes.append(df)

    house_df = pd.concat(house_all_dataframes, ignore_index=True)
    
    estate_df = estate_df.drop(['Unnamed: 0'],axis = 1)
    house_df = house_df.drop(['Unnamed: 0'],axis = 1)
    
    return estate_df, house_df

### estate 데이터 전처리

In [3]:
def estate_preprocessing(estate_df):
    # 주소 데이터가 없을 경우 해당 행 삭제
    estate_df = estate_df.drop(estate_df[estate_df['주소'].isna()].index)
    estate_df = estate_df.drop(['총주차대수'], axis=1)
    estate_df = estate_df.reset_index(drop = True)
    
    # 지번주소와 도로명 주소로 나누기
    estate_df['address'] = estate_df['주소'].apply(lambda x:str(x).split('\n')[0])
    estate_df['address_road'] = estate_df['주소'].apply(lambda x:str(x).split('\n')[1])
    
    # 구와 동 컬럼 생성
    estate_df['SGG_NM'] = estate_df['address'].apply(lambda x:x.split()[1])
    estate_df['BJDONG_NM'] = estate_df['address'].apply(lambda x:x.split()[2])
    
    return estate_df

### house 데이터와 estate 데이터 합친 후 전처리

In [4]:
def house_preprocessing(all_df):
    all_df['해당층/총층'] = all_df['해당층/총층'].astype('str')
    all_df['방수/욕실수'] = all_df['방수/욕실수'].astype('str')
    all_df['해당층'] = all_df['해당층/총층'].apply(lambda x:x.split('/')[0]+'층' if x.split('/')[0]!='nan' else np.NaN)
    all_df['총층'] = all_df['해당층/총층'].apply(lambda x:x.split('/')[1] if x.split('/')[0]!='nan' else np.NaN)
    all_df['방수'] = all_df['방수/욕실수'].apply(lambda x:x.split('/')[0]+'개' if x.split('/')[0]!='nan' else np.NaN)
    all_df['욕실수'] = all_df['방수/욕실수'].apply(lambda x:x.split('/')[1] if x.split('/')[0]!='nan' else np.NaN)
    all_df = all_df.drop(['해당층/총층','방수/욕실수'], axis=1)
    
    all_df['price'] = all_df['price'].astype('str')
    price_list = list(all_df['price'])
    new_price_list = []

    for price in price_list:
        price_res = ''
        for p in price:
            if p.isdigit():
                price_res += p
        if '억' in price and price.split('억')[1]=='':
            new_price_list.append(int(price_res+'0000'))
        else:
            new_price_list.append(int(price_res))
            
    all_df['OBJ_AMT'] = new_price_list
    all_df = all_df.drop(['price'],axis=1)
    
    return all_df

In [5]:
def house_estate_merge_preprocessing(house_df, estate_df):
    house_df['BLDG_NM'] = house_df['building_name'].apply(lambda x:x.split()[0])
    house_df = house_df.drop(['building_name'], axis=1)
    
    all_df = pd.merge(house_df, estate_df, left_on = ['BLDG_NM','gu','dong'], right_on = ['estate_name','SGG_NM','BJDONG_NM'], how = 'left')
    
    all_df = all_df.drop(all_df[all_df['SGG_NM'].isna()].index)
    all_df = all_df.reset_index(drop = True)
    
    all_df = house_preprocessing(all_df)
    
    return all_df

In [6]:
apt_estate_df, apt_df = data_load('APT')
opst_estate_df, opst_df = data_load('OPST')

apt_estate_df = estate_preprocessing(apt_estate_df)
opst_estate_df = estate_preprocessing(opst_estate_df)

apt_all_df = house_estate_merge_preprocessing(apt_df,apt_estate_df)
opst_all_df = house_estate_merge_preprocessing(opst_df,opst_estate_df)

# 아파트 추가 전처리
drop_column_list = ['건축물 용도','gu','dong','estate_code_x','매물특징','기보증금/월세','난방(방식/연료)','입주가능일','매물번호','중개사','관리비 포함','융자금','직거래 정보','estate_code_y','estate_name','세대수', '저/최고층', '사용승인일','용적률','건폐율', '건설사', '난방', '관리사무소','주소', '면적', '수도배관교체','latitude','longitude']
apt_all_df = apt_all_df.drop(drop_column_list, axis=1)
apt_all_df['공급/전용면적'] = apt_all_df['공급/전용면적'].astype('str')
apt_all_df['공급면적'] = apt_all_df['공급/전용면적'].apply(lambda x:x.split('/')[0] if x.split('/')[0]!='nan' else np.NaN)
apt_all_df['전용면적'] = apt_all_df['공급/전용면적'].apply(lambda x:x.split('/')[1] if x.split('/')[0]!='nan' else np.NaN)
apt_all_df = apt_all_df.drop(['공급/전용면적'],axis=1)

# 오피스텔 추가 전처리
drop_column_list = ['건축물 용도','중개사','gu','dong','estate_code_x', '매물특징','용도', '방구조','복층여부','관리비 포함', '기보증금/월세','현관구조', '난방(방식/연료)', '입주가능일','매물번호','latitude', 'longitude','직거래 정보','estate_code_y', 'estate_name', '세대수', '저/최고층', '사용승인일', '용적률', '건폐율', '건설사', '난방', '관리사무소', '주소', '면적']
opst_all_df = opst_all_df.drop(drop_column_list, axis=1)
opst_all_df['계약/전용면적'] = opst_all_df['계약/전용면적'].astype('str')
opst_all_df['공급면적'] = opst_all_df['계약/전용면적'].apply(lambda x:x.split('/')[0] if x.split('/')[0]!='nan' else np.NaN)
opst_all_df['전용면적'] = opst_all_df['계약/전용면적'].apply(lambda x:x.split('/')[1] if x.split('/')[0]!='nan' else np.NaN)
opst_all_df = opst_all_df.drop(['계약/전용면적'],axis=1)


In [7]:
apt_all_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92460 entries, 0 to 92459
Data columns (total 19 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   방향            88025 non-null  object
 1   현관구조          88025 non-null  object
 2   총주차대수         88025 non-null  object
 3   해당면적 세대수      88025 non-null  object
 4   매물설명          58096 non-null  object
 5   house_type    92460 non-null  object
 6   월관리비          76416 non-null  object
 7   BLDG_NM       92460 non-null  object
 8   address       92460 non-null  object
 9   address_road  92460 non-null  object
 10  SGG_NM        92460 non-null  object
 11  BJDONG_NM     92460 non-null  object
 12  해당층           88025 non-null  object
 13  총층            88025 non-null  object
 14  방수            88025 non-null  object
 15  욕실수           88025 non-null  object
 16  OBJ_AMT       92460 non-null  int64 
 17  공급면적          88025 non-null  object
 18  전용면적          88025 non-null  object
dtypes: i

In [8]:
opst_all_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16300 entries, 0 to 16299
Data columns (total 18 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   월관리비          14076 non-null  object
 1   방향            15844 non-null  object
 2   총주차대수         15844 non-null  object
 3   해당면적 세대수      15844 non-null  object
 4   매물설명          12981 non-null  object
 5   house_type    16300 non-null  object
 6   BLDG_NM       16300 non-null  object
 7   address       16300 non-null  object
 8   address_road  16300 non-null  object
 9   SGG_NM        16300 non-null  object
 10  BJDONG_NM     16300 non-null  object
 11  해당층           15844 non-null  object
 12  총층            15844 non-null  object
 13  방수            15844 non-null  object
 14  욕실수           15844 non-null  object
 15  OBJ_AMT       16300 non-null  int64 
 16  공급면적          15844 non-null  object
 17  전용면적          15844 non-null  object
dtypes: int64(1), object(17)
memory usage: 2.2+ MB


In [9]:
apt_all_df

,방향,현관구조,총주차대수,해당면적 세대수,매물설명,house_type,월관리비,BLDG_NM,address,address_road,SGG_NM,BJDONG_NM,해당층,총층,방수,욕실수,OBJ_AMT,공급면적,전용면적
0,남동향(거실 기준),계단식,3961대(세대당 1.72대),99세대,"◆ 개포 래미안포레스트 ◆\n\n-총 2,296세대 31개동\n-22평, 25평, ...",아파트,NaN,개포래미안포레스트,서울시 강남구 개포동 1282,서울시 강남구 개포로 264,강남구,개포동,12층,35층,4개,2개,380000,128.91㎡,96.93㎡(전용률75%)
1,남동향(거실 기준),계단식,3961대(세대당 1.72대),99세대,"개포지구 재건축 전문, 드림부동산(02-572-3111)은 개포동에서 자란 토박이 ...",아파트,30만원,개포래미안포레스트,서울시 강남구 개포동 1282,서울시 강남구 개포로 264,강남구,개포동,29층,33층,4개,2개,340000,128.91㎡,96.93㎡(전용률75%)
2,남동향(거실 기준),계단식,3961대(세대당 1.73대),30세대,▣개포동의 모든것! 오랜 전통의 새방 단지내부동산입니다▣\n◈단지내 중심부에 위치한...,아파트,20만원,개포래미안포레스트,서울시 강남구 개포동 1282,서울시 강남구 개포로 264,강남구,개포동,16층,33층,3개,2개,270000,115.18㎡,84.59㎡(전용률73%)
3,남동향(거실 기준),계단식,3961대(세대당 1.72대),266세대,♣♣♣ 행운부동산입니다 ♣♣♣\n\n- 개포 래미안포레스트 매매/전세/월세 \n\n...,아파트,25만원,개포래미안포레스트,서울시 강남구 개포동 1282,서울시 강남구 개포로 264,강남구,개포동,12층,33층,3개,2개,260000,115.37㎡,84.83㎡(전용률74%)
4,남향(거실 기준),계단식,3961대(세대당 1.72대),30세대,♣♣♣ 행운부동산입니다 ♣♣♣\n\n- 개포 래미안포레스트 매매/전세/월세 \n\n...,아파트,30만원,개포래미안포레스트,서울시 강남구 개포동 1282,서울시 강남구 개포로 264,강남구,개포동,16층,33층,3개,2개,270000,115.18㎡,84.59㎡(전용률73%)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92455,남향(거실 기준),계단식,17대(세대당 1대),6세대,7호선 중화역 도보10분거리 정도 이용가능한 \n전용넓은 초급매물 엘리베이터 있는 ...,아파트,7만원,한영,서울시 중랑구 중화동 274-51,서울시 중랑구 동일로144길 74,중랑구,중화동,4층,7층,3개,2개,53000,96.01㎡,84.69㎡(전용률88%)
92456,서향(거실 기준),계단식,17대(세대당 1.1대),1세대,"3룸, 거실, 화장실 1개 및 넓은 통베란다(ㄱ자) 구조로 \n2021년 7월 S급...",아파트,7만원,한영,서울시 중랑구 중화동 274-51,서울시 중랑구 동일로144길 74,중랑구,중화동,고층,7층,3개,1개,40000,65.36㎡,57.66㎡(전용률88%)
92457,북향(거실 기준),계단식,19대(세대당 1대),1세대,7호선 중화역 도보 8분 역세권으로 강남에 직장 있으신 분들 출퇴근 용이합니다.\n...,아파트,7만원,한영,서울시 중랑구 중화동 274-51,서울시 중랑구 동일로144길 74,중랑구,중화동,고층,7층,3개,2개,66000,123.02㎡,104.93㎡(전용률85%)
92458,남향(거실 기준),계단식,14대(세대당 14대),5세대,1.인근 시세보다 저렴한 급매 아파트 입니다.\n1.구조와 채광상태 양호합니다.\n...,아파트,NaN,현대휴앤미,서울시 중랑구 중화동 11,서울시 중랑구 봉화산로27가길 23,중랑구,중화동,3층,7층,3개,2개,55000,115.32㎡,100.2㎡(전용률87%)


## 빌라

In [10]:
villa_df = pd.read_csv('./data/Villa/naver_selenium_villa_info.csv')

In [11]:
villa_df = villa_df.drop(['Unnamed: 0'],axis = 1)

In [12]:
drop_column_list = ['주차가능여부','총세대수','latitude', 'longitude','매물특징','대지지분', '용적률/건폐율','입주가능일', '융자금', '기보증금/월세','난방(방식/연료)','사용승인일','용도지역', '건축물 용도', '주구조', '매물번호','중개사', '대지/연면적', '건축/전용면적', '지상층/지하층', '복층여부', '재건축/재개발', '월관리비','관리비 포함', '직거래 정보', '준공인가일']
villa_df = villa_df.drop(drop_column_list, axis = 1)
villa_df

,building_name,price,gu,dong,house_type,소재지,공급/전용면적,해당층/총층,방수/욕실수,관리비,방향,총주차대수,매물설명
0,다가구,"11억 5,000",강남구,개포동,단독/다가구,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,빌라 누리찬,"12억 5,000",강남구,개포동,빌라,서울시 강남구 개포동 169-8,35.15㎡/28.49㎡(전용률81%),4/4층,3/2개,7만원 상세보기,남동향(안방 기준),5대(세대당 0.50대),◆재개발 사업성 최상 지역입니다!\n◆ 사무실 방문하시면 사업성 상세히\n설명 드립...
2,다가구,40억,강남구,개포동,단독/다가구,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,빌라 누리찬,11억,강남구,개포동,빌라,서울시 강남구 개포동 169-8,30.84㎡/25㎡(전용률81%),4/4층,3/2개,7만원 상세보기,남동향(안방 기준),5대(세대당 0.50대),◆재개발 사업성 최상 지역입니다!\n◆ 사무실 방문하시면 사업성 상세히\n 설명 드...
4,빌라 누리찬빌라,"8억 4,000",강남구,개포동,빌라,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
688,빌라,"4억 6,700",중랑구,망우동,빌라,서울시 중랑구 망우동 396-41,54.34㎡/45.96㎡(전용률85%),4/5층,3/2개,0원,남향(거실 기준),10대(세대당 0.83대),【 망우동 신축빌라 매매 】\n\n♣우림시장 인근 신축쓰리룸\n\n▣구조 : 방3개...
689,단독,8억,중랑구,망우동,단독/다가구,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
690,빌라,"4억 7,500",중랑구,망우동,빌라,서울시 중랑구 망우동 396-35,63.32㎡/55.53㎡(전용률88%),중/5층,3/2개,확인 어려움\n도움말,동향(거실 기준),8대(세대당 0.80대),【 망우동 신축빌라 3룸 매매 】\n\n◎ 입주 : 즉시입주\n◎ 구조 : 방3 거...
691,빌라,"3억 8,800",중랑구,망우동,빌라,서울시 중랑구 망우동 462-19,58.64㎡/49.59㎡(전용률85%),중/5층,3/2개,5만원 상세보기,남동향(거실 기준),8대(세대당 1.00대),----------------------------------------------...


In [13]:
villa_df = house_preprocessing(villa_df)
villa_df

,building_name,gu,dong,house_type,소재지,공급/전용면적,관리비,방향,총주차대수,매물설명,해당층,총층,방수,욕실수,OBJ_AMT
0,다가구,강남구,개포동,단독/다가구,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,115000
1,빌라 누리찬,강남구,개포동,빌라,서울시 강남구 개포동 169-8,35.15㎡/28.49㎡(전용률81%),7만원 상세보기,남동향(안방 기준),5대(세대당 0.50대),◆재개발 사업성 최상 지역입니다!\n◆ 사무실 방문하시면 사업성 상세히\n설명 드립...,4층,4층,3개,2개,125000
2,다가구,강남구,개포동,단독/다가구,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,400000
3,빌라 누리찬,강남구,개포동,빌라,서울시 강남구 개포동 169-8,30.84㎡/25㎡(전용률81%),7만원 상세보기,남동향(안방 기준),5대(세대당 0.50대),◆재개발 사업성 최상 지역입니다!\n◆ 사무실 방문하시면 사업성 상세히\n 설명 드...,4층,4층,3개,2개,110000
4,빌라 누리찬빌라,강남구,개포동,빌라,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,84000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
688,빌라,중랑구,망우동,빌라,서울시 중랑구 망우동 396-41,54.34㎡/45.96㎡(전용률85%),0원,남향(거실 기준),10대(세대당 0.83대),【 망우동 신축빌라 매매 】\n\n♣우림시장 인근 신축쓰리룸\n\n▣구조 : 방3개...,4층,5층,3개,2개,46700
689,단독,중랑구,망우동,단독/다가구,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,80000
690,빌라,중랑구,망우동,빌라,서울시 중랑구 망우동 396-35,63.32㎡/55.53㎡(전용률88%),확인 어려움\n도움말,동향(거실 기준),8대(세대당 0.80대),【 망우동 신축빌라 3룸 매매 】\n\n◎ 입주 : 즉시입주\n◎ 구조 : 방3 거...,중층,5층,3개,2개,47500
691,빌라,중랑구,망우동,빌라,서울시 중랑구 망우동 462-19,58.64㎡/49.59㎡(전용률85%),5만원 상세보기,남동향(거실 기준),8대(세대당 1.00대),----------------------------------------------...,중층,5층,3개,2개,38800


In [14]:
def villa_name_preprocessing(name):
    name_list = name.split()
    if len(name_list) == 1:
        return name_list[0]
    else:
        if '중개수수료' in name_list[1] or '분양' in name_list[1]:
            return name_list[0]
        else:
            return name_list[1]

In [15]:
villa_df['BLDG_NM'] = villa_df['building_name'].apply(villa_name_preprocessing)

In [16]:
villa_df.loc[villa_df['building_name'].apply(lambda x:x.split()[0]=='연립'), 'house_type'] = '연립다세대'

In [17]:
villa_df['공급/전용면적'] = villa_df['공급/전용면적'].astype('str')
villa_df['공급면적'] = villa_df['공급/전용면적'].apply(lambda x:x.split('/')[0] if x.split('/')[0]!='nan' else np.NaN)
villa_df['전용면적'] = villa_df['공급/전용면적'].apply(lambda x:x.split('/')[1] if x.split('/')[0]!='nan' else np.NaN)

In [18]:
villa_df.columns

Index(['building_name', 'gu', 'dong', 'house_type', '소재지', '공급/전용면적', '관리비',
       '방향', '총주차대수', '매물설명', '해당층', '총층', '방수', '욕실수', 'OBJ_AMT', 'BLDG_NM',
       '공급면적', '전용면적'],
      dtype='object')

In [19]:
villa_df = villa_df.drop(['building_name','공급/전용면적'],axis=1)

## 건물들 모두 합치기

In [20]:
villa_df = villa_df.rename(columns={'방향':'DIRECTION',
'소재지':'ADDRESS',
'관리비':'MANAGEMENT_FEE',
'gu':'SGG_NM',
'dong':'BJDONG_NM',
'총주차대수':'PARKING_SPACES',
'매물설명':'DESCRIBTION',
'해당층': 'FLOOR',
'총층':'TOTAL_FLOOR',
'방수':'ROOM',
'욕실수':'BATHROOM',
'공급면적':'SUPPLY_AREA',
'전용면적':'NET_LEASABLE_AREA',
'house_type':'HOUSE_TYPE'
})

In [21]:
apt_all_df = apt_all_df.rename(columns={'방향':'DIRECTION',
'현관구조':'ENTRANCE_STRUCTURE',
 '해당면적 세대수':'NUMBER_OF_HOUSEHOLDS',
'address':'ADDRESS',
'address_road':'ADDRESS_ROAD',
'월관리비':'MANAGEMENT_FEE',
'gu':'SGG_NM',
'dong':'BJDONG_NM',
'총주차대수':'PARKING_SPACES',
'매물설명':'DESCRIBTION',
'해당층': 'FLOOR',
'총층':'TOTAL_FLOOR',
'방수':'ROOM',
'욕실수':'BATHROOM',
'공급면적':'SUPPLY_AREA',
'전용면적':'NET_LEASABLE_AREA',
'house_type':'HOUSE_TYPE'
})

In [22]:
opst_all_df = opst_all_df.rename(columns={'방향':'DIRECTION',
'현관구조':'ENTRANCE_STRUCTURE',
 '해당면적 세대수':'NUMBER_OF_HOUSEHOLDS',
'address':'ADDRESS',
'address_road':'ADDRESS_ROAD',
'월관리비':'MANAGEMENT_FEE',
'gu':'SGG_NM',
'dong':'BJDONG_NM',
'총주차대수':'PARKING_SPACES',
'매물설명':'DESCRIBTION',
'해당층': 'FLOOR',
'총층':'TOTAL_FLOOR',
'방수':'ROOM',
'욕실수':'BATHROOM',
'공급면적':'SUPPLY_AREA',
'전용면적':'NET_LEASABLE_AREA',
'house_type':'HOUSE_TYPE'
})

In [23]:
villa_df.columns

Index(['SGG_NM', 'BJDONG_NM', 'HOUSE_TYPE', 'ADDRESS', 'MANAGEMENT_FEE',
       'DIRECTION', 'PARKING_SPACES', 'DESCRIBTION', 'FLOOR', 'TOTAL_FLOOR',
       'ROOM', 'BATHROOM', 'OBJ_AMT', 'BLDG_NM', 'SUPPLY_AREA',
       'NET_LEASABLE_AREA'],
      dtype='object')

In [24]:
apt_all_df.columns

Index(['DIRECTION', 'ENTRANCE_STRUCTURE', 'PARKING_SPACES',
       'NUMBER_OF_HOUSEHOLDS', 'DESCRIBTION', 'HOUSE_TYPE', 'MANAGEMENT_FEE',
       'BLDG_NM', 'ADDRESS', 'ADDRESS_ROAD', 'SGG_NM', 'BJDONG_NM', 'FLOOR',
       'TOTAL_FLOOR', 'ROOM', 'BATHROOM', 'OBJ_AMT', 'SUPPLY_AREA',
       'NET_LEASABLE_AREA'],
      dtype='object')

In [25]:
opst_all_df.columns

Index(['MANAGEMENT_FEE', 'DIRECTION', 'PARKING_SPACES', 'NUMBER_OF_HOUSEHOLDS',
       'DESCRIBTION', 'HOUSE_TYPE', 'BLDG_NM', 'ADDRESS', 'ADDRESS_ROAD',
       'SGG_NM', 'BJDONG_NM', 'FLOOR', 'TOTAL_FLOOR', 'ROOM', 'BATHROOM',
       'OBJ_AMT', 'SUPPLY_AREA', 'NET_LEASABLE_AREA'],
      dtype='object')

In [26]:
total_df = pd.concat([villa_df,apt_all_df,opst_all_df],ignore_index=True)
total_df

,SGG_NM,BJDONG_NM,HOUSE_TYPE,ADDRESS,MANAGEMENT_FEE,DIRECTION,PARKING_SPACES,DESCRIBTION,FLOOR,TOTAL_FLOOR,ROOM,BATHROOM,OBJ_AMT,BLDG_NM,SUPPLY_AREA,NET_LEASABLE_AREA,ENTRANCE_STRUCTURE,NUMBER_OF_HOUSEHOLDS,ADDRESS_ROAD
0,강남구,개포동,단독/다가구,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,115000,다가구,NaN,NaN,NaN,NaN,NaN
1,강남구,개포동,빌라,서울시 강남구 개포동 169-8,7만원 상세보기,남동향(안방 기준),5대(세대당 0.50대),◆재개발 사업성 최상 지역입니다!\n◆ 사무실 방문하시면 사업성 상세히\n설명 드립...,4층,4층,3개,2개,125000,누리찬,35.15㎡,28.49㎡(전용률81%),NaN,NaN,NaN
2,강남구,개포동,단독/다가구,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,400000,다가구,NaN,NaN,NaN,NaN,NaN
3,강남구,개포동,빌라,서울시 강남구 개포동 169-8,7만원 상세보기,남동향(안방 기준),5대(세대당 0.50대),◆재개발 사업성 최상 지역입니다!\n◆ 사무실 방문하시면 사업성 상세히\n 설명 드...,4층,4층,3개,2개,110000,누리찬,30.84㎡,25㎡(전용률81%),NaN,NaN,NaN
4,강남구,개포동,빌라,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,84000,누리찬빌라,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109448,중랑구,신내동,오피스텔,서울시 중랑구 신내동 644,4만원,동향(거실 기준),31대(세대당 0.4대),★★ 네이버 검색 '뉴리얼티부동산'(SNS)에 유용한 정보 참조가능 ★★\n\n♣ ...,3층,5층,1개,1개,6500,엘리어트하우스,21.09㎡,11.88㎡(전용률56%),NaN,1세대,서울시 중랑구 신내로 193
109449,중랑구,중화동,오피스텔,서울시 중랑구 중화동 208-4,8만원,남향(거실 기준),35대(세대당 0.32대),중랑역1분 대로변 빌트인형 풀옵션 오피스텔 수익형 매물 전용 약7평,7층,15층,1개,1개,11000,중화범양프레체,46.02㎡,22.21㎡(전용률48%),NaN,108세대,서울시 중랑구 중랑천로14길 58
109450,중랑구,중화동,오피스텔,서울시 중랑구 중화동 208-4,7.5만원,남향(거실 기준),35대(세대당 0.32대),전세금 1억1천만원 이번에 새로 전세계약 갱신했습니다.,4층,15층,1개,1개,11000,중화범양프레체,46.02㎡,22.21㎡(전용률48%),NaN,108세대,서울시 중랑구 중랑천로14길 58
109451,중랑구,중화동,오피스텔,서울시 중랑구 중화동 208-4,NaN,남동향(거실 기준),86대(세대당 0.6대),"- 버스정류장 도보 3분, 편리한 대중교통수단 이용\n- 재래시장, 은행, 스타벅스...",6층,15층,1개,1개,11800,중화범양프레체,46.02㎡,22.21㎡(전용률48%),NaN,108세대,서울시 중랑구 중랑천로14길 58


In [143]:
total_df.to_csv('./data/crawling_total_data.csv', index=False)

In [ ]:
#DESCRIBTION 특수 문자 제거